# Чтение данных

In [1]:
import pandas as pd

from tqdm import tqdm
tqdm.pandas()

df_cleansed = pd.read_csv("df_cleansed.csv", sep=";")
df_cleansed.shape

(3371478, 3)

# Лемматизация

*Дальнейшая кластеризация строится на предположении, что первое существительное в заголовке столбца отражает ту сущность, которая в нем содержится*

## Работа с подвыборкой, из-за нехватки ресурсов

In [2]:
df_cleansed_sample = df_cleansed.sample(random_state=2023, frac=0.02)
df_cleansed_sample

,table_id,column_id,column_name
2720148,dataset/Dataset/data/3483326/table_6_meta.json,1,алгоритм шифрования
2167367,dataset/Dataset/data/5318820/table_1_meta.json,0,чарт
2438329,dataset/Dataset/data/2077534/table_3_meta.json,1,легендас года
2649773,dataset/Dataset/data/1603484/table_2_meta.json,1,титулы по меступроведенияматчей турнира
1782388,dataset/Dataset/data/3057834/table_6_meta.json,3,яванцы
...,...,...,...
1525628,dataset/Dataset/data/4403233/table_0_meta.json,0,год
643388,dataset/Dataset/data/4166863/table_0_meta.json,0,условные цветовые обозначения
3181561,dataset/Dataset/data/6592321/table_0_meta.json,2,русское название
313452,dataset/Dataset/data/2977964/table_0_meta.json,0,год


## 3. Построение леммы для первого слова заголовка

In [3]:
import spacy
nlp = spacy.load("ru_core_news_sm", disable=['parser', 'ner'])

In [4]:
def lemmatize_first_word(sequence: str) -> str:
    for token in nlp(sequence):
        if token.pos_ == "NOUN":
            return token.lemma_
    return None

df_cleansed_sample["first_noun"] = df_cleansed_sample["column_name"].progress_apply(lemmatize_first_word)

100%|████████████████████████████████████| 67430/67430 [01:28<00:00, 762.64it/s]


In [5]:
df_cleansed_sample["first_noun"].isna().sum()

3593

In [6]:
df_cleansed_sample[df_cleansed_sample["first_noun"].isna()]['column_name'].unique()

array(['откууда', 'режиссры', 'европа', 'режисср', 'жлтый', 'пропущено',
       'четвртый', 'запасной', 'первый', 'нац', 'погибло', 'зубные',
       'том', 'конечные остановкиаб', 'всего', 'символшлефли', 'третий',
       'испанский', 'за что получена', 'дома', 'фил', 'итого',
       'квалифицирована как', 'я луна', 'эргативный', 'таксон', 'куда',
       'франция', 'площадькм', 'спущен', 'партнр', 'ф и о', 'азотное',
       'вен', 'патронус', 'исландия', 'амк', 'передние', 'и после',
       'авестийский', 'вицегубернаторанглрусскк', 'синонимы', 'третья',
       'гласные', 'соправители', 'ничьи', 'мсб', 'горная', 'начало',
       'задненбные', 'самый результативный', 'ранено', 'шел', 'нов',
       'когда использовался', 'налт', 'второй', 'я пада', 'вступил',
       'населениечел', 'сша', 'побит', 'умер', 'хун', 'произведено',
       'дополнительно', 'независимые', 'крп', 'видзв', 'синий',
       'винительный', 'платформаы', 'нушл', 'построено', 'австралия',
       'номинирован', 'русски

# Создание эмбеддингов
Для кластеризации удаляем столбцы, для которых не было получено ембеддингов

In [7]:
import gensim.downloader
import numpy as np

ru_model = gensim.downloader.load("word2vec-ruscorpora-300")

In [8]:
def get_embedding(word: str) -> list|None:
    try:
        return ru_model.__getitem__([f"{word}_NOUN"])[0].astype(float)
    except KeyError:
        return None

df_cleansed_sample["embedding"] = df_cleansed_sample["first_noun"].progress_apply(get_embedding)
df_cleansed_sample

100%|█████████████████████████████████| 67430/67430 [00:00<00:00, 235130.03it/s]


,table_id,column_id,column_name,first_noun,embedding
2720148,dataset/Dataset/data/3483326/table_6_meta.json,1,алгоритм шифрования,алгоритм,"[0.04933843016624451, -0.04191359877586365, -0..."
2167367,dataset/Dataset/data/5318820/table_1_meta.json,0,чарт,чарт,"[0.029721805825829506, 0.03733756020665169, -0..."
2438329,dataset/Dataset/data/2077534/table_3_meta.json,1,легендас года,легендас,None
2649773,dataset/Dataset/data/1603484/table_2_meta.json,1,титулы по меступроведенияматчей турнира,титул,"[0.12037932872772217, -0.047469932585954666, -..."
1782388,dataset/Dataset/data/3057834/table_6_meta.json,3,яванцы,яванец,"[0.0452362522482872, -0.030167412012815475, -0..."
...,...,...,...,...,...
1525628,dataset/Dataset/data/4403233/table_0_meta.json,0,год,год,"[-0.02274102345108986, -0.043927066028118134, ..."
643388,dataset/Dataset/data/4166863/table_0_meta.json,0,условные цветовые обозначения,обозначение,"[0.07376829534769058, 0.031282924115657806, -0..."
3181561,dataset/Dataset/data/6592321/table_0_meta.json,2,русское название,название,"[0.15615154802799225, 0.042191389948129654, -0..."
313452,dataset/Dataset/data/2977964/table_0_meta.json,0,год,год,"[-0.02274102345108986, -0.043927066028118134, ..."


## Фильтрация столбцов, для которых не построились эмбеддинги

In [9]:
mask_hasnt_embedding = df_cleansed_sample["embedding"].isna()
df_cleansed_sample = df_cleansed_sample[~mask_hasnt_embedding]
df_cleansed_sample = df_cleansed_sample.drop_duplicates(subset=["first_noun"]) 
df_cleansed_sample

,table_id,column_id,column_name,first_noun,embedding
2720148,dataset/Dataset/data/3483326/table_6_meta.json,1,алгоритм шифрования,алгоритм,"[0.04933843016624451, -0.04191359877586365, -0..."
2167367,dataset/Dataset/data/5318820/table_1_meta.json,0,чарт,чарт,"[0.029721805825829506, 0.03733756020665169, -0..."
2649773,dataset/Dataset/data/1603484/table_2_meta.json,1,титулы по меступроведенияматчей турнира,титул,"[0.12037932872772217, -0.047469932585954666, -..."
1782388,dataset/Dataset/data/3057834/table_6_meta.json,3,яванцы,яванец,"[0.0452362522482872, -0.030167412012815475, -0..."
3370842,dataset/Dataset/data/1176153/table_0_meta.json,3,полное население,население,"[0.032919786870479584, -0.030262669548392296, ..."
...,...,...,...,...,...
2318256,dataset/Dataset/data/184282/table_1_meta.json,2,без подпора в облицованном канале,подпор,"[0.013514978811144829, 0.04030410572886467, -0..."
2446801,dataset/Dataset/data/6625005/table_0_meta.json,5,белорусы тыс,белорус,"[0.01209559291601181, 0.09944429993629456, 0.0..."
51431,dataset/Dataset/data/3630710/table_6_meta.json,3,девочки,девочка,"[-0.09372813254594803, 0.060462430119514465, -..."
737424,dataset/Dataset/data/3732326/table_0_meta.json,0,файловый менеджер,менеджер,"[-0.027020612731575966, -0.027391137555241585,..."


# Кластеризация

---

https://github.com/src-d/kmcuda

TODO: HDBSCAN

---

## Metrics

### Silhouette-coefficient

https://scikit-learn.org/stable/modules/clustering.html#silhouette-coefficient

Более высокая оценка коэффициента относится к модели с лучше определенными кластерами.

Коэффициент силуэт определен для каждой точки и состоит из двух оценок:

$a$ - среднее расстояние между  и всеми другими точками в кластере

$b$ - среднее расстояние между точкой и всеми другими точками в другом ближайшем кластере

Для конкретной точки, коэффициент рассчитывается:

$s = \dfrac{b - a}{max(a, b)}$

Для оценки кластеризации используется среднее из всех значений коэффициента для каждой точки.

*Advantages:*
- Оценка находится на отрезке [-1; 1], где -1 для некорректных кластеров, +1 для очень плотных кластеров. Оценки около нуля указывают на пересекающиеся кластеры.
- Оценка выше, когда кластеры плотные и хорошо разделены

*Drawbacks:*
- Коэффициент обычно выше для выпуклых кластеров, чем для других концепций кластеров, таких как кластеры, основанные на плотности (DBSCAN)

In [10]:
from sklearn import metrics

def silhouette_score(X, labels, metric="euclidean"):
    return metrics.silhouette_score(X, labels, metric=metric)

### Calinski-Harabasz index

https://scikit-learn.org/stable/modules/clustering.html#calinski-harabasz-index

Индекс представляет собой отношение суммы межкластерной дисперсии и внутрикластерной дисперсии для всех кластеров (где дисперсия определяется как сумма квадратов расстояний):

Для набора данных $S$ размера $n_E$, который был сгруппирован в $k$ кластеров, оценка Калински-Харабаша определяется как соотношение среднего значения межкластерной дисперсии и внутрикластерной дисперсии:

$s = \dfrac{tr(B_k)}{tr(W_k)} * \dfrac{n_e - k}{k - 1}$

где $tr(B_k)$ является сумма элементов главной диагонали матрицы межгрупповой дисперсии и $tr(W_k)$ — сумма элементов главной диагонали матрицы внутрикластерной дисперсии:

$W_k = \sum_{q=1}^{k}\sum_{x \in C_q}{(x-c_q)(x-c_q)^T}$

$B_k = \sum_{q=1}^{k}n_q*(c_q-c_E)(c_q-c_E)^T$

$C_q$ набор точек в кластере $q$, $c_q$ - центр кластера $q$, $c_E$ - центр $E$, $n_q$ - число точек в кластере $q$.

Advantages:
- Оценка выше, когда кластеры плотные и хорошо разделены
- Оценка быстро вычисляется

Drawbacks:
- Коэффициент обычно выше для выпуклых кластеров, чем для других концепций кластеров, таких как кластеры, основанные на плотности (DBSCAN)

In [11]:
def calinski_harabasz_index(X, labels):
    return metrics.calinski_harabasz_score(X, labels)

### Davies-Bouldin index

https://scikit-learn.org/stable/modules/clustering.html#davies-bouldin-index

Оценка интерпретируется, как среднее сходство между кластерами, где сходство — это мера, которая сравнивает расстояние между кластерами с размер самих кластеров.

Минимальная оценка = 0. Значения близкие к нулю, указывают на лучшее разбиение.

Индекс определяется как среднее сходство между каждым кластером $C_i, i = 1, ..., k$ и самым похожим на него $C_j$. В контексте этом показателе сходство определяется как мера $R_{ij}$:

$R_{ij} = \dfrac{s_i + s_j}{d_{ij}}$

где:

$s_i$ среднее расстояние между каждой точкой кластера $i$ и центра тяжести этого кластера.

$d_{ij}$ расстояние между центрами кластеров $i$ и $j$.


$DB = \dfrac{1}{k} * \sum_{i=1}^{k} \max_{i \neq j} R{ij}$

Преимущества:
- Расчет показателей проще, чем расчет показателей Silhouette.
- Индекс основывается исключительно на величинах и характеристиках, присущих набору данных, поскольку при его расчете используются только расстояния между точками.

Недостатки:
- Коэффициент обычно выше для выпуклых кластеров, чем для других концепций кластеров, таких как кластеры, основанные на плотности (DBSCAN)
- Использование расстояния до центроида ограничивает метрику расстояния евклидовым пространством.



In [12]:
def davies_bouldin_index(X, labels):
    return metrics.davies_bouldin_score(X, labels)

### Get metrics values

In [13]:
def get_metrics(X, labels):
    metrics = dict()
    metrics["silhouette_score"] = silhouette_score(X, labels)
    metrics["calinski_harabasz_index"] = calinski_harabasz_index(X, labels)
    metrics["davies_bouldin_index"] = davies_bouldin_index(X, labels)

    return metrics

## Ram info

In [14]:
import os

sample = True

mem_bytes = os.sysconf('SC_PAGE_SIZE') * os.sysconf('SC_PHYS_PAGES')
ram = mem_bytes / (1024. ** 3)
ram

14.983638763427734

## Reshape embeddings

In [15]:
import numpy as np
from random import randint

rs = 13

embeddings_reshaped = np.array(list(map(lambda x: x, df_cleansed_sample["embedding"])))
embeddings_reshaped.shape

df_cls = pd.DataFrame()
df_cls["first_noun"] = df_cleansed_sample["first_noun"]

In [16]:
def get_cluster_id_words(cluster_name: str):
    cluster_to_word = dict()
    for cluster_id in df_cls[cluster_name].unique():
        cluster_to_word[cluster_id] = df_cls[df_cls[name] == cluster_id]["first_noun"].unique()
    
    return cluster_to_word

def print_cluster_to_word(cluster_dict: dict, threshold: int = 1):
    for i in cluster_dict.keys():
        if len(cluster_dict[i]) > threshold:
            print(i, cluster_dict[i], len(cluster_dict[i]))

## DBSCAN; memory complexity($N^2$)
https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html


Алгоритм DBSCAN рассматривает кластеры как области высокой плотности. разделены областями с низкой плотностью. Из-за этого довольно общего взгляда кластеры найденное с помощью DBSCAN, может иметь любую форму, в отличие от k-средних, которое предполагает, что кластеры имеют выпуклую форму. 

Алгоритм DBSCAN является детерминированным и всегда генерирует одни и те же кластеры. когда даны одни и те же данные в том же порядке. Однако результаты могут отличаться, когда данные предоставляются в другом порядке.

Текущая реализация использует ball-trees и kd-trees.

Эта реализация неэффективна с точки зрения использования памяти, на больших датасетах лучше использовать OPTICS или HDBSCAN

In [17]:
if sample or ram > 20:
    from sklearn.cluster import DBSCAN
    
    name = "dbscan"
    model = DBSCAN
    
    df_cls[name] = model(eps=0.9, min_samples=2).fit_predict(embeddings_reshaped)

    cluster_dict = get_cluster_id_words(name)
    print_cluster_to_word(cluster_dict)
else:
    print("Not enough memory")

0 ['алгоритм' 'показатель' 'способ' 'динамика' 'использование'
 'характеристика' 'прирост' 'уровень' 'величина' 'оценка' 'метод'
 'классификация' 'индекс' 'критерий' 'компонент' 'размерность' 'параметр'
 'ввп' 'эффективность' 'коэффициент' 'наличие' 'технология'
 'результативность' 'техника' 'применение' 'отсутствие' 'анализ'
 'исследование'] 28
-1 ['чарт' 'яванец' 'категория' ... 'подпор' 'менеджер' 'рок'] 1432
1 ['титул' 'звание' 'должность' 'чин'] 4
2 ['население' 'житель'] 2
3 ['матч' 'мяч' 'кубок' 'спортсмен' 'турнир' 'игрок' 'соревнование' 'финал'
 'пилот' 'поул' 'чемпион' 'сборная' 'чемпионат' 'игра' 'старт' 'финиш'
 'суперкубок' 'атлет' 'теннисист' 'тренер' 'спринт' 'вратарь'
 'спортсменка' 'гонка' 'фигурист' 'заезд' 'гонщик' 'футболист'
 'нападающий' 'полузащитник' 'первенство' 'динамо' 'чемпионка' 'велоспорт'
 'ралли' 'юниор' 'форвард' 'шайба' 'спорт' 'полуфинал' 'плейофф'
 'бундеслига' 'биатлонист' 'фигуристка' 'олимпиада' 'гимнастка'
 'состязание' 'овертайм' 'спартак' 'хокк

In [18]:
get_metrics(embeddings_reshaped, df_cls[name])

{'silhouette_score': -0.06462939856511825,
 'calinski_harabasz_index': 2.9887848074062586,
 'davies_bouldin_index': 1.9067114563621759}

In [19]:
len(df_cls['dbscan'].unique())

152

## Affinity propagation; memory comp.($N^2$), time comp.($N^2 * T$)

AffinityPropagation создает кластеры, посылая сообщения между парами образцов до сходимости. Затем набор данных описывается с помощью небольшого числа образцов, которые определяются как наиболее репрезентативные для других образцов. 

Сообщения, отправляемые между парами, представляют собой оценку пригодности одного образца быть образцом другого, которая обновляется в зависимости от значений, полученных от других пар. 

Это обновление происходит итеративно до сходимости, после чего выбираются окончательные образцы, и, следовательно, получается окончательная кластеризация.

In [20]:
if sample or ram > 50:
    from sklearn.cluster import AffinityPropagation
    
    name = "affinity_prop"
    model = AffinityPropagation
    df_cls[name] = model().fit_predict(embeddings_reshaped)

    cluster_dict = get_cluster_id_words(name)
    print_cluster_to_word(cluster_dict)
else:
    print("Not enough memory")

101 ['алгоритм' 'модель' 'координата' 'характеристика' 'объект' 'модификация'
 'константа' 'диапазон' 'ксп' 'компонент' 'стандарт' 'фактор'
 'размерность' 'параметр' 'вероятность' 'техпроцесс' 'вариант' 'разбиение'
 'спецификация' 'периодичность' 'вариация' 'кластер' 'зависимость'
 'распределение' 'диаграмма' 'шкала' 'точность' 'пропорциональность'
 'отображение'] 29
173 ['чарт' 'лейбл' 'релиз' 'сингл' 'трек' 'дискография' 'саундтрек' 'хит'
 'мультиплекс' 'тихомир' 'жаргонизм' 'ремикс' 'лэйбл' 'слоган' 'наив'
 'мантра'] 16
4 ['титул' 'звание' 'должность' 'ранг' 'гражданство' 'карьера' 'присвоение'
 'разряд' 'чин' 'кавалер' 'профессия' 'сан'] 12
193 ['яванец' 'уйгур' 'кореец' 'кантон' 'айн' 'приамурье' 'китаец'] 7
24 ['население' 'регион' 'статус' 'муниципалитет' 'субъект' 'субрегион'
 'освоенность' 'ресурс' 'кузбасс' 'федерация' 'миграция' 'атр' 'ингушетия'] 13
188 ['категория' 'транскрипция' 'обозначение' 'транслитерация' 'префикс' 'мас'
 'принадлежность' 'единица' 'артикль' 'падеж' '

## BIRCH
https://scikit-learn.org/stable/modules/generated/sklearn.cluster.Birch.html

Birch строит (Clustering Feature Tree, CFT), для заданных данных. 

Данные сжимаются с потерями до набора узлов кластеризации признаков (CF Nodes). Узлы CF имеют ряд подкластеров, называемых подкластерами признаков кластеризации (CF Subclusters), и эти подкластеры CF, расположенные в нетерминальных узлах CF, могут иметь узлы CF в качестве дочерних.

Подкластеры CF содержат необходимую для кластеризации информацию, которая позволяет избежать необходимости хранить в памяти все входные данные.

Алгоритм BIRCH имеет два параметра - порог и фактор ветвления. Фактор ветвления ограничивает количество подкластеров в узле, а порог - расстояние между входящей выборкой и существующими подкластерами.

Если n_clusters равно None, то подкластеры из листьев считываются напрямую, в противном случае шаг глобальной кластеризации помечает эти подкластеры в глобальные кластеры (метки), и образцы сопоставляются с глобальной меткой ближайшего подкластера.

In [21]:
def find_best_clustering_params(X, model_cls, min_num_clusters=100, max_num_clusters=101):
    best_num_clusters = -1
    best_clustering_score = -5
    best_clusters = None

    from tqdm import tqdm
    # tqdm
    for num_clusters in tqdm(range(min_num_clusters, max_num_clusters)):
        labels = model(n_clusters=num_clusters).fit_predict(embeddings_reshaped)
        score = calinski_harabasz_index(X, labels)
        # score = silhouette_score(X, labels)
        # score = davies_bouldin_index(X, labels)

        if score > best_clustering_score:
            best_clustering_score = score
            best_num_clusters = num_clusters
            best_clusters = labels

    return {
        "best_clustering_score": best_clustering_score,
        "best_num_clusters": best_num_clusters,
        "best_clusters": best_clusters
    }


In [22]:
from sklearn.cluster import Birch

name = "birch"
model = Birch

clustering_params = find_best_clustering_params(embeddings_reshaped, model, min_num_clusters=200, max_num_clusters=220)

print(clustering_params, end="\n\n\n")

df_cls[name] = clustering_params["best_clusters"]
cluster_dict = get_cluster_id_words(name)

print_cluster_to_word(cluster_dict)

100%|███████████████████████████████████████████| 20/20 [00:06<00:00,  2.91it/s]

{'best_clustering_score': 5.077229933890018, 'best_num_clusters': 200, 'best_clusters': array([ 48, 169, 143, ..., 151,   6, 169])}


48 ['алгоритм' 'показатель' 'модель' 'система' 'таблица' 'динамика' 'метод'
 'механизм' 'индекс' 'критерий' 'размерность' 'параметр' 'индикатор'
 'техпроцесс' 'разбиение' 'спецификация' 'схема' 'рис' 'кластер'
 'конструкция' 'диаграмма' 'шкала'] 22
169 ['чарт' 'лейбл' 'релиз' 'сингл' 'дискография' 'саундтрек' 'хит' 'ремикс'
 'лэйбл' 'хард' 'рок'] 11
143 ['титул' 'звание' 'должность' 'назначение' 'ранг' 'пост' 'служба' 'чин'
 'отставка'] 9
182 ['яванец' 'македония' 'мусульманин' 'белград' 'шах' 'араб' 'султан'
 'болгарин' 'вилайет' 'турок' 'серб' 'курд' 'нахия' 'перс' 'бей'] 15
60 ['население' 'численность' 'млн' 'число' 'сумма' 'масса' 'количество'
 'процент' 'микрограмм' 'мощность' 'км' 'доля' 'кг' 'тыс' 'тонна'
 'минимум' 'руб' 'среднее' 'миллион' 'вклад' 'меньшинство' 'сотня'
 'тысяча' 'большинство' 'максимум'] 25
54 ['категория' 'группа' 'класс' 'сос

In [23]:
get_metrics(embeddings_reshaped, df_cls[name])

{'silhouette_score': 0.03368078649065845,
 'calinski_harabasz_index': 5.077229933890018,
 'davies_bouldin_index': 2.7805275651323145}

## OPTICS; time complexity($N^2$)

Алгоритм OPTICS имеет много общего с алгоритмом DBSCAN и может рассматриваться как обобщение DBSCAN, в котором требование к параметру eps ослаблено с единственного значения до диапазона значений. 

Ключевое различие между DBSCAN и OPTICS заключается в том, что алгоритм OPTICS строит граф достижимости, который присваивает каждому образцу как расстояние_достижимости, так и место в атрибуте_упорядочивания_кластера; эти два атрибута присваиваются при подгонке модели и используются для определения принадлежности к кластеру. 

In [24]:
if sample:
    from sklearn.cluster import OPTICS

    name = "optics"
    model = OPTICS
    df_cls[name] = model(
        min_samples=2,
        max_eps=0.91,
        min_cluster_size=2,
        n_jobs=-1,
        # cluster_method="dbscan"
    ).fit_predict(embeddings_reshaped)
    
    cluster_dict = get_cluster_id_words(name)
    print_cluster_to_word(cluster_dict)
else:
    print("Do not wait slow algos enabled")

-1 ['алгоритм' 'чарт' 'яванец' ... 'подпор' 'менеджер' 'рок'] 1693
13 ['титул' 'звание' 'должность' 'чин'] 4
14 ['население' 'житель'] 2
15 ['матч' 'турнир' 'чемпион' 'сборная' 'чемпионат' 'тренер' 'чемпионка'] 7
28 ['название' 'наименование'] 2
29 ['очки' 'очко'] 2
30 ['страна' 'регион' 'правительство' 'муниципалитет' 'государство'
 'губернатор' 'сенатор' 'власть' 'империя' 'администрация' 'советник'] 11
31 ['имя' 'фамилия'] 2
32 ['соперник' 'соперница' 'конкурент'] 3
36 ['сертификация' 'сертификат'] 2
37 ['итог' 'результат'] 2
38 ['победитель' 'победа' 'победительница'] 3
0 ['показатель' 'параметр'] 2
6 ['млн' 'тыс'] 2
39 ['планета' 'астероид'] 2
40 ['город' 'столица' 'провинция' 'пригород'] 4
42 ['факультет' 'колледж' 'университет' 'кафедра' 'ректор'] 5
43 ['армия' 'фронт' 'дивизия' 'войско' 'полк' 'эскадрон'] 6
50 ['отель' 'мажестик'] 2
51 ['деталь' 'подробность'] 2
52 ['судьба' 'участь'] 2
3 ['способ' 'метод'] 2
53 ['владелец' 'собственник'] 2
54 ['награда' 'медаль'] 2
21 ['лауреа

## Agglomerative Hierarchy

Иерархическая кластеризация - это общее семейство алгоритмов кластеризации, которые строят вложенные кластеры путем последовательного слияния или разделения. Эта иерархия кластеров представляется в виде дерева (или дендрограммы). Корень дерева - это уникальный кластер, в котором собраны все образцы, а листья - кластеры, в которых есть только один образец.

In [25]:
if sample or ram > 16:
    from sklearn.cluster import AgglomerativeClustering

    num_clusters = 100
    
    name = "agglomerative"
    model = AgglomerativeClustering

    df_cls[name] = model().fit_predict(embeddings_reshaped)

    cluster_dict = get_cluster_id_words(name)
    print_cluster_to_word(cluster_dict)
else:
    print("Not enough memory")

0 ['алгоритм' 'чарт' 'титул' ... 'признание' 'инцидент' 'менеджер'] 1183
1 ['яванец' 'население' 'матч' 'позиция' 'мяч' 'очки' 'дата' 'кубок' 'место'
 'сезон' 'соперник' 'линия' 'год' 'спортсмен' 'турнир' 'мон' 'соперница'
 'команда' 'победитель' 'игрок' 'лек' 'нагрузка' 'кан' 'клуб' 'планета'
 'город' 'соревнование' 'армия' 'расстояние' 'отель' 'слалом' 'рио'
 'судьба' 'расположение' 'финал' 'пол' 'поз' 'владелец' 'лига' 'пункт'
 'цвет' 'ита' 'пилот' 'поул' 'конь' 'корпус' 'двигатель' 'раунд' 'тур'
 'координата' 'чемпион' 'сплит' 'масса' 'бомбардировщик' 'очко' 'материал'
 'район' 'блок' 'пассажиропоток' 'фра' 'месяц' 'тор' 'кат' 'улица' 'брак'
 'сборная' 'противник' 'заплыв' 'остров' 'напор' 'вва' 'победа'
 'чемпионат' 'палочка' 'юрг' 'игра' 'штурмовка' 'медалист' 'маршрут'
 'метр' 'фронт' 'вместимость' 'старт' 'финиш' 'орбита' 'градус' 'дорожка'
 'суперкубок' 'церемония' 'рывок' 'пантера' 'илл' 'судно' 'вооружение'
 'восхождение' 'атлет' 'пас' 'капитан' 'вершина' 'пар' 'доска' 'сеп'

## K-means

Алгоритм KMeans кластеризует данные, пытаясь разделить образцы по n группы с одинаковой дисперсией, минимизируя критерий - внутрикластерная сумма квадратов. Для этого алгоритма требуется число кластеров, которые необходимо указать. Он хорошо масштабируется для большого количества выборок и имеет использовался в самых разных областях применения в самых разных областях.

In [26]:
import warnings
warnings.filterwarnings('ignore')

In [27]:
from sklearn.cluster import KMeans

name = "kmeans"
model = KMeans

clustering_params = find_best_clustering_params(embeddings_reshaped, model, min_num_clusters=145, max_num_clusters=155)

print(clustering_params, end="\n\n\n")

df_cls[name] = clustering_params["best_clusters"]
cluster_dict = get_cluster_id_words(name)

print_cluster_to_word(cluster_dict)

100%|███████████████████████████████████████████| 10/10 [00:13<00:00,  1.37s/it]

{'best_clustering_score': 5.764167414303667, 'best_num_clusters': 145, 'best_clusters': array([21, 62, 44, ..., 19, 50, 62], dtype=int32)}


21 ['алгоритм' 'модель' 'координата' 'идентификатор' 'модификация'
 'процессор' 'сервер' 'размерность' 'множитель' 'уравнение' 'модуль'
 'комбинация' 'пиктограмма' 'дисплей' 'вариант' 'дау' 'разбиение'
 'спецификация' 'фокусировка' 'последовательность' 'пользователь' 'схема'
 'кластер' 'локация' 'диаграмма' 'отображение'] 26
62 ['чарт' 'лейбл' 'серия' 'сайт' 'релиз' 'сингл' 'альбом' 'дискография'
 'хит' 'ремикс' 'рок'] 11
44 ['титул' 'имя' 'наименование' 'звание' 'должность' 'награда' 'ранг'
 'статус' 'гражданство' 'медаль' 'карьера' 'присвоение' 'разряд' 'чин'
 'честь' 'фамилия' 'профессия' 'сан' 'достоинство' 'прозвище'] 20
18 ['яванец' 'уйгур' 'киргиз' 'кореец' 'кантон' 'айн' 'таджик' 'казах'
 'китаец'] 9
14 ['население' 'численность' 'итог' 'показатель' 'рост' 'процент'
 'рождаемость' 'динамика' 'прирост' 'рейтинг' 'уровень'
 'продолжительност

In [28]:
get_metrics(embeddings_reshaped, df_cls[name])

{'silhouette_score': 0.025343803621024036,
 'calinski_harabasz_index': 5.764167414303667,
 'davies_bouldin_index': 3.1783457538898254}

## Mini batch k-means
Берется батч (срез) данных, по ним строится k-means, за счет чего быстро работает)

In [29]:
from sklearn.cluster import MiniBatchKMeans

name = "minibatch-kmeans"
model = MiniBatchKMeans

clustering_params = find_best_clustering_params(embeddings_reshaped, model,min_num_clusters=150, max_num_clusters=200)

print(clustering_params, end="\n\n\n")

df_cls[name] = clustering_params["best_clusters"]
cluster_dict = get_cluster_id_words(name)

print_cluster_to_word(cluster_dict)

100%|███████████████████████████████████████████| 50/50 [00:19<00:00,  2.52it/s]

{'best_clustering_score': 5.523038984910519, 'best_num_clusters': 153, 'best_clusters': array([ 65,  24, 120, ...,  22,  82,  24], dtype=int32)}


65 ['алгоритм' 'сплит' 'блок' 'дст' 'идентификатор' 'кегль' 'код'
 'обозначение' 'формат' 'информация' 'оператор' 'вис' 'сеть' 'процессор'
 'сервер' 'расшифровка' 'нумерация' 'приставка' 'синхронизация'
 'видеокарта' 'размерность' 'модуль' 'диск' 'аббревиатура' 'домен' 'ввод'
 'тос' 'кодировка' 'пиктограмма' 'дисплей' 'дау' 'разбиение'
 'спецификация' 'фокусировка' 'пользователь' 'абонент' 'память' 'пси'
 'локация' 'байт' 'бит' 'логотип' 'набор' 'запись' 'озу' 'монтаж'
 'линейка' 'метка' 'диаграмма' 'табло' 'хостинг' 'хеш' 'инн' 'эоп'
 'отображение'] 55
24 ['чарт' 'шоу' 'номинант' 'подиум' 'лейбл' 'релиз' 'бонус' 'сингл'
 'продюсер' 'спонсор' 'клип' 'скип' 'финалистка' 'фестиваль' 'солид'
 'боулер' 'трек' 'топ' 'мультсериал' 'видеоклип' 'дискография' 'саундтрек'
 'кинокомпания' 'хит' 'популярность' 'мультиплекс' 'ремикс' 'лэйбл' 'хард'
 'тел

In [30]:
get_metrics(embeddings_reshaped, df_cls[name])

{'silhouette_score': 0.007550441738885716,
 'calinski_harabasz_index': 5.523038984910519,
 'davies_bouldin_index': 2.8758772495243115}

In [31]:
save_result = False
if save_result:
    df_cls.to_csv("df_cls.csv", sep=";", index=False)